# 60‑Minute Mini Project: Forecast a Seasonal Series

**Goal:** Beat a seasonal‑naive baseline with Exponential Smoothing (ETS) and report MAE/RMSE.

**Deliverables:**
- Plot showing train/test, seasonal‑naive, and ETS forecast
- Table of MAE/RMSE (seasonal‑naive vs ETS)
- 3–5 bullet takeaways

Dataset: Monthly CO2 (built into `statsmodels`).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import statsmodels.api as sm
from statsmodels.tsa.holtwinters import ExponentialSmoothing

rmse = lambda y, yhat: mean_squared_error(y, yhat, squared=False)

## 1) Load & visualize

In [ ]:
co2 = sm.datasets.co2.load_pandas().data
co2['date'] = pd.to_datetime(co2['date'])
s = co2.set_index('date')['co2'].resample('M').mean().ffill()
ax = s.plot(title='Monthly CO2 (ppm)'); ax.set_ylabel('ppm'); plt.show()

## 2) Time‑aware split + seasonal‑naive baseline

In [ ]:
split = int(len(s)*0.8)
train, test = s.iloc[:split], s.iloc[split:]
season = 12
naive = s.shift(1).loc[test.index]
snaive = s.shift(season).loc[test.index]
print('Naive   MAE/RMSE:', mean_absolute_error(test, naive), rmse(test, naive))
print('S‑Naive MAE/RMSE:', mean_absolute_error(test, snaive), rmse(test, snaive))
ax = train.plot(label='train')
test.plot(ax=ax, label='test')
snaive.plot(ax=ax, label='seasonal‑naive')
plt.legend(); plt.show()

## 3) ETS forecast and compare

In [ ]:
ets = ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=season)
fit = ets.fit()
fc = fit.forecast(len(test))
mae_ets = mean_absolute_error(test, fc)
rmse_ets = rmse(test, fc)
print('ETS     MAE/RMSE:', mae_ets, rmse_ets)
ax = train.plot(label='train')
test.plot(ax=ax, label='test')
fc.plot(ax=ax, label='ETS forecast')
plt.legend(); plt.show()
import pandas as pd
res = pd.DataFrame({'Model':['Seasonal‑Naive','ETS (additive)'], 'MAE':[mean_absolute_error(test, snaive), mae_ets], 'RMSE':[rmse(test, snaive), rmse_ets]})
res

## 4) Notes
- ETS beats seasonal‑naive on MAE/RMSE by __%.
- Clear yearly seasonality; additive components worked well.
- Next: try log transform or multiplicative seasonality if variance grows with level.